# Estimators in Tensorflow
We are going to learn plain and simple estimators which can make linear regression simpler and easier

## The Basic Usage of Estimator

In [1]:
#Out packages
import tensorflow as tf
import numpy as np

#Checking the version
print(tf.__version__)

1.3.0


In [6]:
# Declare list of features. We only have one numeric feature. There are many
# other types of columns that are more complicated and useful.
feature_columns = [tf.feature_column.numeric_column('x', shape=[1])]

### What is an Estimator :
An estimator is the front end to invoke training (fitting) and evaluation(inference). There are many predefined types like linear regression, linear classification, and many neural network classifiers and regressors. The following code provides an estimator that does linear regression.

In [19]:
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

#TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2, 5, 8, 1])
y_eval = np.array([-1.01, -4.1, -7, 0.])

input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/wl/n2_2rwc93zz1xy58jjxcvq5m0000gn/T/tmpkeqrokuy', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [20]:
# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/wl/n2_2rwc93zz1xy58jjxcvq5m0000gn/T/tmpkeqrokuy/model.ckpt.
INFO:tensorflow:loss = 11.0, step = 1
INFO:tensorflow:global_step/sec: 530.434
INFO:tensorflow:loss = 0.1992, step = 101 (0.192 sec)
INFO:tensorflow:global_step/sec: 757.438
INFO:tensorflow:loss = 0.0146594, step = 201 (0.132 sec)
INFO:tensorflow:global_step/sec: 951.113
INFO:tensorflow:loss = 0.0106635, step = 301 (0.106 sec)
INFO:tensorflow:global_step/sec: 958.882
INFO:tensorflow:loss = 0.00185864, step = 401 (0.104 sec)
INFO:tensorflow:global_step/sec: 930.294
INFO:tensorflow:loss = 7.15146e-05, step = 501 (0.107 sec)
INFO:tensorflow:global_step/sec: 801.449
INFO:tensorflow:loss = 8.16655e-05, step = 601 (0.126 sec)
INFO:tensorflow:global_step/sec: 806.067
INFO:tensorflow:loss = 1.73338e-05, step = 701 (0.122 sec)
INFO:tensorflow:global_step/sec: 830.277
INFO:tensorflow:loss = 8.94877e-07, step = 801 (0.121 sec)
INFO:tens

In [21]:
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Starting evaluation at 2017-09-08-10:22:40
INFO:tensorflow:Restoring parameters from /var/folders/wl/n2_2rwc93zz1xy58jjxcvq5m0000gn/T/tmpkeqrokuy/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-09-08-10:22:41
INFO:tensorflow:Saving dict for global step 1000: average_loss = 2.41913e-08, global_step = 1000, loss = 9.67652e-08
INFO:tensorflow:Starting evaluation at 2017-09-08-10:22:41
INFO:tensorflow:Restoring parameters from /var/folders/wl/n2_2rwc93zz1xy58jjxcvq5m0000gn/T/tmpkeqrokuy/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-09-08-10:22:42
INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.00254011, global_step = 1000, loss = 0.0101604
train metrics: {'average_loss': 2.4191312e-08, 'loss': 9.6765248e-08, 'global_step': 1000}
eval metrics: {'average_loss': 0.0025401104, 'loss': 0.010160442, 'global_step': 1000}


## Custom Model 

In [22]:
import numpy as np
import tensorflow as tf

In [25]:
# Declare list of features, we only have one real-valued feature
def model_fn(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W * features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
    # EstimatorSpec connects subgraphs we built to the
  # appropriate functionality.
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=y,
      loss=loss,
      train_op=train)

In [26]:
estimator = tf.estimator.Estimator(model_fn=model_fn)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/wl/n2_2rwc93zz1xy58jjxcvq5m0000gn/T/tmp7sd30r69', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [27]:
# train
estimator.train(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/wl/n2_2rwc93zz1xy58jjxcvq5m0000gn/T/tmp7sd30r69/model.ckpt.
INFO:tensorflow:loss = 8.66328258808, step = 1
INFO:tensorflow:global_step/sec: 867.913
INFO:tensorflow:loss = 0.0355069538472, step = 101 (0.119 sec)
INFO:tensorflow:global_step/sec: 635.336
INFO:tensorflow:loss = 0.00187912138361, step = 201 (0.158 sec)
INFO:tensorflow:global_step/sec: 544.505
INFO:tensorflow:loss = 0.000208277648644, step = 301 (0.183 sec)
INFO:tensorflow:global_step/sec: 924.118
INFO:tensorflow:loss = 1.89639229428e-05, step = 401 (0.108 sec)
INFO:tensorflow:global_step/sec: 873.753
INFO:tensorflow:loss = 1.93501599463e-06, step = 501 (0.112 sec)
INFO:tensorflow:global_step/sec: 632.639
INFO:tensorflow:loss = 2.52058855581e-07, step = 601 (0.159 sec)
INFO:tensorflow:global_step/sec: 660.938
INFO:tensorflow:loss = 3.87570816478e-08, step = 701 (0.151 sec)
INFO:tensorflow:global_step/sec: 615.271
INFO:tenso